In [1]:
import pandas as pd 
import numpy as np
import copy
import datetime
import os
import glob
import copy
import datetime
from struct import *
from pandas.tseries.offsets import Hour, Minute, DateOffset, Day
import matplotlib.pyplot as plt

### 1.1  现在日线选取成交量的高位点

In [2]:
data = pd.read_csv('D:/tushare/data/day/sz399300 .csv',sep = ',',encoding='utf8')

In [3]:
data.head()

,Unnamed: 0,date,code,open,high,low,close,amount,vol
0,0,2008-01-02 14:55:00,sz399300,5349.76,5404.93,5283.45,5385.10,9.888643e+09,45668710
1,1,2008-01-03 14:55:00,sz399300,5381.15,5422.67,5315.95,5422.03,1.359834e+10,64645878
2,2,2008-01-04 14:55:00,sz399300,5430.63,5499.08,5422.46,5483.65,1.188662e+10,51746420
3,3,2008-01-07 14:55:00,sz399300,5480.44,5569.15,5455.50,5556.59,1.231896e+10,53485409
4,4,2008-01-08 14:55:00,sz399300,5575.95,5630.62,5485.23,5528.05,1.479757e+10,64899501


In [4]:
#data['datetime'] = data['date']+ ' '+ data['time']
data.index = pd.to_datetime(data['date'])
#data1.drop(['datetime',],axis=1,inplace=True)
data.drop(['code','amount','code','Unnamed: 0','date'],axis=1,inplace=True)

In [5]:
def addMA(data, p=144):
    #不使用TA-LIB函数库MA_Type: 0=SMA, 1=EMA, 2=WMA, 3=DEMA, 4=TEMA, 5=TRIMA, 6=KAMA, 7=MAMA, 8=T3 (Default=SMA)
    #data['MA5'] = ta.MA(data.close.values,5,matype=0)
    #data['MA144'] = ta.MA(data.close.values,144,matype=0)
    data['MA'+str(p)] = data['close'].rolling(p).mean()   
    data['MA'+str(p)+'a'] = data['MA'+str(p)].pct_change()

In [6]:
#w_vol=data['volume'].resample('W',label='right',closed='right').mean().dropna()
#w_vol=pd.DataFrame(w_vol)

In [7]:
addMA(data)

In [8]:
#成交量大于昨天的量和明天的量，即为高点，在‘B’列进行标记
data['B']=np.where((data['vol'] > data['vol'].shift(1)) & (data['vol'] > data['vol'].shift(-1)),1,0)
#将时间设为14:55，方便与15分钟图匹配
data.index = data.index-10*Minute()
data.tail(20)

,open,high,low,close,vol,MA144,MA144a,B
date,,,,,,,,
2019-01-25 14:45:00,3165.18,3203.17,3164.41,3184.47,83967729,3255.302917,-0.000586,1
2019-01-28 14:45:00,3203.12,3228.87,3174.72,3183.78,82622431,3253.637986,-0.000511,0
2019-01-29 14:45:00,3177.26,3200.99,3148.23,3193.97,77658334,3251.436528,-0.000677,0
2019-01-30 14:45:00,3181.66,3202.90,3167.89,3168.48,63746517,3249.773472,-0.000511,0
2019-01-31 14:45:00,3182.70,3216.07,3178.01,3201.63,86247839,3248.331458,-0.000444,1
2019-02-01 14:45:00,3225.73,3247.50,3207.72,3247.40,82019641,3247.523472,-0.000249,0
2019-02-11 14:45:00,3239.81,3307.06,3239.81,3306.47,98818434,3247.273681,-0.000077,0
2019-02-12 14:45:00,3306.35,3335.36,3298.63,3330.34,122232462,3247.032153,-0.000074,0
2019-02-13 14:45:00,3337.78,3405.58,3329.14,3397.03,163828332,3246.600556,-0.000133,1


In [9]:
data.drop(['open','high','low','close'],axis=1,inplace=True)

In [10]:
data.head()

,vol,MA144,MA144a,B
date,,,,
2008-01-02 14:45:00,45668710,NaN,NaN,0
2008-01-03 14:45:00,64645878,NaN,NaN,1
2008-01-04 14:45:00,51746420,NaN,NaN,0
2008-01-07 14:45:00,53485409,NaN,NaN,0
2008-01-08 14:45:00,64899501,NaN,NaN,1


### 读取通达信5分钟数据，并整理为15分钟数据

In [11]:
min15 = pd.read_csv('D:/tushare/data/5min/sz399300.csv',sep = ',',encoding='utf8')
min15.head()

,Unnamed: 0,date,time,code,open,high,low,close,amount,vol
0,0,2018-01-02,09:35:00,sz399300,4030.860107,4059.600098,4030.860107,4059.600098,8.479816e+09,84798125
1,1,2018-01-02,09:40:00,sz399300,4059.929932,4064.340088,4059.929932,4061.449951,7.243599e+09,72435962
2,2,2018-01-02,09:45:00,sz399300,4061.239990,4062.070068,4055.209961,4055.389893,6.607842e+09,66078393
3,3,2018-01-02,09:50:00,sz399300,4055.169922,4066.810059,4055.169922,4066.810059,6.364881e+09,63648777
4,4,2018-01-02,09:55:00,sz399300,4066.510010,4071.600098,4066.439941,4070.090088,6.838485e+09,68384818


In [12]:
min15['datetime'] = min15['date']+ ' '+ min15['time']
min15.index = pd.to_datetime(min15['datetime'])

In [13]:
min15.tail(10)

,Unnamed: 0,date,time,code,open,high,low,close,amount,vol,datetime
datetime,,,,,,,,,,,
2019-02-28 14:15:00,13430,2019-02-28,14:15:00,sz399300,3665.800049,3668.580078,3665.419922,3668.409912,2.386919e+09,23869167,2019-02-28 14:15:00
2019-02-28 14:20:00,13431,2019-02-28,14:20:00,sz399300,3668.419922,3668.510010,3660.600098,3660.709961,2.665824e+09,26658218,2019-02-28 14:20:00
2019-02-28 14:25:00,13432,2019-02-28,14:25:00,sz399300,3660.969971,3662.909912,3656.860107,3662.439941,3.474227e+09,34742244,2019-02-28 14:25:00
2019-02-28 14:30:00,13433,2019-02-28,14:30:00,sz399300,3662.189941,3662.340088,3659.030029,3662.260010,3.020440e+09,30204362,2019-02-28 14:30:00
2019-02-28 14:35:00,13434,2019-02-28,14:35:00,sz399300,3662.550049,3674.199951,3662.550049,3671.969971,4.147347e+09,41473446,2019-02-28 14:35:00
2019-02-28 14:40:00,13435,2019-02-28,14:40:00,sz399300,3672.100098,3674.520020,3671.629883,3671.629883,3.791323e+09,37913204,2019-02-28 14:40:00
2019-02-28 14:45:00,13436,2019-02-28,14:45:00,sz399300,3671.699951,3672.050049,3663.879883,3664.229980,3.848749e+09,38487460,2019-02-28 14:45:00
2019-02-28 14:50:00,13437,2019-02-28,14:50:00,sz399300,3664.520020,3669.810059,3663.399902,3669.649902,4.098097e+09,40980942,2019-02-28 14:50:00
2019-02-28 14:55:00,13438,2019-02-28,14:55:00,sz399300,3670.100098,3670.100098,3667.370117,3667.479980,5.101355e+09,51013519,2019-02-28 14:55:00


In [14]:
min15.drop(['Unnamed: 0','amount','datetime'],axis=1,inplace=True)

In [15]:
time= min15['time']
date= min15['date']

In [16]:
time = time[2::3] #从第二行开始，每3行取样，把时间改为15分钟周期
date = date[2::3]

In [17]:
min15 = min15.resample('15T',label='right',closed='right').mean().dropna() #整个数组重新取样，15分钟周期，按平均值统计，删除字符列，仅保留数值列

In [18]:
addMA(min15,p=80) #15分钟线的80周期均线和均线的导数为日线图的5日线

In [19]:
min15['time'] = time

In [20]:
min15['date'] = date

### 将日线数据合并在15分钟数据中

In [21]:
df= pd.merge(min15,data,left_index=True,right_index=True,how='left')
#df.drop(['code','amount','time','Unnamed: 0','date'],axis=1,inplace=True)

In [22]:
df.fillna(int(0) ,inplace=True)#用0填充NaN

In [23]:
df

,open,high,low,close,vol_x,MA80,MA80a,time,date,vol_y,MA144,MA144a,B
datetime,,,,,,,,,,,,,
2018-01-02 09:45:00,4050.676676,4062.003418,4048.666667,4058.813314,7.443749e+07,0.000000,0.000000,09:45:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 10:00:00,4063.933350,4071.233398,4063.909993,4069.410075,6.753364e+07,0.000000,0.000000,10:00:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 10:15:00,4065.590088,4065.590088,4060.450033,4062.290039,5.127486e+07,0.000000,0.000000,10:15:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 10:30:00,4067.126628,4072.409993,4067.076660,4072.146566,4.119257e+07,0.000000,0.000000,10:30:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 10:45:00,4075.173340,4077.533366,4073.513346,4075.639974,3.741625e+07,0.000000,0.000000,10:45:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 11:00:00,4079.059977,4080.993327,4076.813314,4079.006673,2.896134e+07,0.000000,0.000000,11:00:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 11:15:00,4077.166667,4079.746745,4076.199951,4078.926676,2.909102e+07,0.000000,0.000000,11:15:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 11:30:00,4083.550049,4084.069987,4081.833333,4082.576660,2.830057e+07,0.000000,0.000000,11:30:00,2018-01-02,0.0,0.000000,0.000000,0.0
2018-01-02 13:15:00,4080.136637,4081.386637,4078.683350,4079.876628,2.829186e+07,0.000000,0.000000,13:15:00,2018-01-02,0.0,0.000000,0.000000,0.0


#### 选出日线图上的成交量高点日

In [24]:
date=df[df['B']==1]['date']#选出高点日，并存入date中

In [25]:
for i in range(len(date)):     #该部分为重点思路
    df.loc[date[i],'B'] = np.ones(16).astype(int) #在15分钟图上，当天的高点标识位“B”全部改为1，记住这种编程思路

In [26]:
df[df['B'] ==1]

,open,high,low,close,vol_x,MA80,MA80a,time,date,vol_y,MA144,MA144a,B
datetime,,,,,,,,,,,,,
2018-01-03 09:45:00,4091.776611,4097.439860,4091.356608,4095.699951,7.407853e+07,0.000000,0.000000,09:45:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 10:00:00,4105.149902,4109.246582,4103.763184,4108.016764,6.716828e+07,0.000000,0.000000,10:00:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 10:15:00,4112.483236,4117.373372,4112.026530,4117.013346,5.745935e+07,0.000000,0.000000,10:15:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 10:30:00,4126.193359,4129.669922,4124.709961,4128.066732,4.967582e+07,0.000000,0.000000,10:30:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 10:45:00,4131.716797,4136.036784,4130.976725,4135.906738,4.531621e+07,0.000000,0.000000,10:45:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 11:00:00,4135.596517,4137.033203,4132.886556,4134.206868,3.774275e+07,0.000000,0.000000,11:00:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 11:15:00,4132.496745,4133.236654,4129.663249,4130.459961,3.326225e+07,0.000000,0.000000,11:15:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 11:30:00,4132.773275,4136.290039,4132.290039,4135.880046,3.131291e+07,0.000000,0.000000,11:30:00,2018-01-03,0.0,0.000000,0.000000,1.0
2018-01-03 13:15:00,4135.583333,4136.270020,4133.223470,4133.516602,3.306695e+07,0.000000,0.000000,13:15:00,2018-01-03,0.0,0.000000,0.000000,1.0


In [27]:
df['C'] = int (0) #在表中增加进场标识位‘0’
df.rename(columns={"vol_x":'V15'},inplace = True)
df.shape

(4480, 14)

In [28]:
#begin = pd.to_datetime(df.iloc[0]['date'])   #表中起始日期
#end   = pd.to_datetime(df.iloc[-1]['date'])   #表中的结束日期
#delta = end - begin                         #表中的总天数，少一天  ，开头减末尾，等于总天数-1

In [29]:
#j= str(begin.year)+'-'+str(begin.month)+'-'+str(begin.day)

#### 以天作为循环单位，每次循环比较当天的16位15分钟数据,#### 核心代码

In [30]:
df

,open,high,low,close,V15,MA80,MA80a,time,date,vol_y,MA144,MA144a,B,C
datetime,,,,,,,,,,,,,,
2018-01-02 09:45:00,4050.676676,4062.003418,4048.666667,4058.813314,7.443749e+07,0.000000,0.000000,09:45:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 10:00:00,4063.933350,4071.233398,4063.909993,4069.410075,6.753364e+07,0.000000,0.000000,10:00:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 10:15:00,4065.590088,4065.590088,4060.450033,4062.290039,5.127486e+07,0.000000,0.000000,10:15:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 10:30:00,4067.126628,4072.409993,4067.076660,4072.146566,4.119257e+07,0.000000,0.000000,10:30:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 10:45:00,4075.173340,4077.533366,4073.513346,4075.639974,3.741625e+07,0.000000,0.000000,10:45:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 11:00:00,4079.059977,4080.993327,4076.813314,4079.006673,2.896134e+07,0.000000,0.000000,11:00:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 11:15:00,4077.166667,4079.746745,4076.199951,4078.926676,2.909102e+07,0.000000,0.000000,11:15:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 11:30:00,4083.550049,4084.069987,4081.833333,4082.576660,2.830057e+07,0.000000,0.000000,11:30:00,2018-01-02,0.0,0.000000,0.000000,0.0,0
2018-01-02 13:15:00,4080.136637,4081.386637,4078.683350,4079.876628,2.829186e+07,0.000000,0.000000,13:15:00,2018-01-02,0.0,0.000000,0.000000,0.0,0


In [31]:
df.shape

(4480, 14)

In [37]:
dfstd = df['2019-1-18']
dfstd.drop(['open', 'high', 'low', 'close', 'MA80', 'MA80a',  'date', 'vol_y', 'MA144', 'MA144a', 'B', 'C'],axis=1,inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
dfstd

,V15,time
datetime,,
2019-01-18 09:45:00,4.152801e+07,09:45:00
2019-01-18 10:00:00,3.227654e+07,10:00:00
2019-01-18 10:15:00,3.384132e+07,10:15:00
2019-01-18 10:30:00,2.032437e+07,10:30:00
2019-01-18 10:45:00,1.993395e+07,10:45:00
2019-01-18 11:00:00,1.753336e+07,11:00:00
2019-01-18 11:15:00,1.543738e+07,11:15:00
2019-01-18 11:30:00,1.405093e+07,11:30:00
2019-01-18 13:15:00,1.342798e+07,13:15:00


In [39]:
n=0
for i in range(5,len(df),16): #从每天的第n个时间点算，周期为16，即15分钟图的1天
    j= df.iloc[i,8]           #读出当天的日期值
    if (df[j].loc[j+' 10:45:00','B']==0) & (n==0): #先寻找成交量标准日，如果不是则直接进入下一天
        continue
    elif(df[j].loc[j+' 10:45:00','B']==1) & (n==0):  #判断当天的日期是否是成交量的高位日
        dfstd = df[j]
        dfstd.drop(['open', 'high', 'low', 'close', 'MA80', 'MA80a',  'date', 'vol_y', 'MA144', 'MA144a', 'B', 'C'],axis=1,inplace=True)
        n=n+1
    elif df[j].loc[j+' 10:45:00','B']==0:
        df_1 = df[j]
        for i in range(16):
            if df_1.iloc[i,4] > dfstd.iloc[i,0]:#寻找第一个突破位
                gold = df_1.iloc[i,8] +' '+df_1.iloc[0,7]#，将该突破位的日期和时刻记入在gold中
                df.loc[gold,'C'] = 1
                break
    elif df[j].loc[j+' 10:45:00','B']==1:
        df_1 = df[j]
        for i in range(16):
            if df_1.iloc[i,4] > dfstd.iloc[i,0]:#寻找第一个突破位
                gold = df_1.iloc[i,8] +' '+df_1.iloc[0,7]#，将该突破位的日期和时刻记入在gold中
                df.loc[gold,'C'] = 1
                break
        dfstd=df[j]
        dfstd.drop(['open', 'high', 'low', 'close', 'MA80', 'MA80a',  'date', 'vol_y', 'MA144', 'MA144a', 'B', 'C'],axis=1,inplace=True)      


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### 出标识点csv文件

In [41]:
linshi= df[df['C']==1]

In [42]:
linshi.to_csv('D:/tushare/sz399300buy.csv',sep = ',',encoding='utf8')#导出标识点csv文件

In [ ]:
dftsd.drop(['open', 'high','low','close' ,'amount','MA80','MA80a','vol_y', 'MA144', 'MA144a','B','C'],axis=1,inplace=True)

In [ ]:
dftsd

In [ ]:
df1 = pd.merge(df1,dftsd,on='time')

In [ ]:
df1.loc[4,'C']

In [ ]:
type(df1)

In [ ]:
dfstd = df[df.iloc[1]['date']]['vol_x']

In [ ]:
dfstd